__собираем предложения по квартирам__

_Evgeny S. Borisov <parser@mechanoid.su>_

In [1]:
# !pacman -S firefox firefox-i18n-r  geckodriver
# !pip install seleniuam

In [2]:
# import re
import sys
import logging
from datetime import datetime as dtm
from tqdm.notebook import tqdm
import pandas as pd

In [3]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.float_format', '{:.2f}'.format)
tqdm.pandas()

logging.basicConfig(
        format=u'[%(levelname)-8s] %(asctime)s | %(message)s',
        datefmt='%Y-%m-%d %H:%M:%S',
        level=logging.INFO,
        # level=logging.DEBUG,
        stream=sys.stdout,
    )

---

In [4]:
# каталог с файлами объявлений
avito_raw_data_path = 'data/raw/flat/avito' 
cian_raw_data_path = 'data/raw/flat/cian'

result_file_path = 'data/data_flat.pkl'

In [5]:
# ts = dtm.now().strftime('%Y-%m-%d_%H-%M')
# ts

## собираем данные авито.ру

In [6]:
import re
from os import listdir

raw_data_files = sorted([
        avito_raw_data_path+'/'+f 
        for f in listdir(avito_raw_data_path) 
        if re.match(r'.+\.xlsx$',f)
    ])
raw_data_files

['data/raw/flat/avito/avito_2022-08-26_17-29_raw.xlsx',
 'data/raw/flat/avito/avito_2022-08-29_13-16_raw.xlsx',
 'data/raw/flat/avito/avito_2022-08-31_17-42_raw.xlsx',
 'data/raw/flat/avito/avito_2022-09-01_14-07_raw.xlsx',
 'data/raw/flat/avito/avito_2022-09-02_13-04_raw.xlsx',
 'data/raw/flat/avito/avito_2022-09-05_14-02_raw.xlsx',
 'data/raw/flat/avito/avito_2022-09-06_13-09_raw.xlsx',
 'data/raw/flat/avito/avito_2022-09-07_12-39_raw.xlsx',
 'data/raw/flat/avito/avito_2022-09-08_13-36_raw.xlsx',
 'data/raw/flat/avito/avito_2022-09-12_13-44_raw.xlsx',
 'data/raw/flat/avito/avito_2022-09-13_13-55_raw.xlsx',
 'data/raw/flat/avito/avito_2022-09-15_10-27_raw.xlsx']

In [8]:
from lib.avito import AvitoDataCleanerRealtyFlat

data_avito = pd.concat([
        pd.read_excel(f)
        .dropna()
        .drop_duplicates(['avito_id','description'])
        .rename(columns={'page':'avito_page'})
        .reset_index(drop=True)  
        for f in raw_data_files
    ]).reset_index(drop=True)

data_avito = AvitoDataCleanerRealtyFlat().transform( data_avito )

print(len(data_avito))

18459


In [9]:
assert len(data_avito)>0

In [10]:
data_avito.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18459 entries, 0 to 18458
Data columns (total 22 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   avito_id       18459 non-null  int64         
 1   title          18459 non-null  object        
 2   price          18459 non-null  int64         
 3   adr            18459 non-null  object        
 4   description    18459 non-null  object        
 5   obj_name       18459 non-null  object        
 6   ts             18459 non-null  datetime64[ns]
 7   avito_page     18459 non-null  int64         
 8   place          18459 non-null  object        
 9   nrooms         18459 non-null  int64         
 10  floor          18459 non-null  int64         
 11  nfloors        18459 non-null  int64         
 12  area           18459 non-null  float64       
 13  is_studio      18459 non-null  bool          
 14  is_apartment   18459 non-null  bool          
 15  is_part        1845

In [11]:
data_avito.isnull().any()

avito_id         False
title            False
price            False
adr              False
description      False
obj_name         False
ts               False
avito_page       False
place            False
nrooms           False
floor            False
nfloors          False
area             False
is_studio        False
is_apartment     False
is_part          False
is_auction       False
is_openspace     False
is_roof          False
is_SNT           False
priceM           False
is_last_floor    False
dtype: bool

In [12]:
# data_avito.query('nrooms==0')
# data_avito[ data_avito['title'].str.len()<1]

## собираем данные циан.ру

In [13]:
import re
from os import listdir

raw_data_files = sorted([
        cian_raw_data_path+'/'+f 
        for f in listdir(cian_raw_data_path) 
        if re.match(r'.+\.pkl$',f)
        #if re.match(r'.+\.xlsx$',f)
    
    ])
raw_data_files

['data/raw/flat/cian/cian_2022-09-08_14-25_raw.pkl',
 'data/raw/flat/cian/cian_2022-09-12_13-59_raw.pkl',
 'data/raw/flat/cian/cian_2022-09-13_13-59_raw.pkl',
 'data/raw/flat/cian/cian_2022-09-15_10-30_raw.pkl']

In [15]:
data_cian = pd.concat([
        pd.read_pickle(f)
         .dropna()
         .drop_duplicates(['LinkArea','Description',])
         .rename(columns={'page':'cian_page'})
         .reset_index(drop=True)  
        for f in raw_data_files
    ]).reset_index(drop=True)

In [16]:
data_cian.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6022 entries, 0 to 6021
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   OfferTitle     6022 non-null   object        
 1   OfferSubtitle  6022 non-null   object        
 2   Deadline       6022 non-null   object        
 3   MainPrice      6022 non-null   object        
 4   PriceInfo      6022 non-null   object        
 5   GeoLabel       6022 non-null   object        
 6   TimeLabel      6022 non-null   object        
 7   LinkArea       6022 non-null   object        
 8   Description    6022 non-null   object        
 9   cian_page      6022 non-null   int64         
 10  ts             6022 non-null   datetime64[ns]
 11  place          6022 non-null   object        
dtypes: datetime64[ns](1), int64(1), object(10)
memory usage: 564.7+ KB


In [17]:
data_cian.isnull().any()

OfferTitle       False
OfferSubtitle    False
Deadline         False
MainPrice        False
PriceInfo        False
GeoLabel         False
TimeLabel        False
LinkArea         False
Description      False
cian_page        False
ts               False
place            False
dtype: bool

In [18]:
swap_title = data_cian['OfferSubtitle'].str.match(r'.*\d+ м², \d+/\d+ эт.*') 

data_cian = pd.concat([
        data_cian[ ~swap_title ],
        data_cian[ swap_title ]
            .rename(columns={'OfferTitle':'OfferSubtitle','OfferSubtitle':'OfferTitle'})
    ]).reset_index(drop=True)


In [19]:
# data_cian

In [20]:
from lib.cian import CianDataCleaner

data_cian = CianDataCleaner().transform( data_cian )

print(len(data_cian))

6022


In [21]:
assert len(data_cian)>0

In [22]:
data_cian.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6022 entries, 0 to 6021
Data columns (total 22 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   title          6022 non-null   object        
 1   obj_name       6022 non-null   object        
 2   adr            6022 non-null   object        
 3   nrooms         6022 non-null   int64         
 4   floor          6022 non-null   int64         
 5   nfloors        6022 non-null   int64         
 6   area           6022 non-null   float64       
 7   is_studio      6022 non-null   bool          
 8   is_apartment   6022 non-null   bool          
 9   is_part        6022 non-null   bool          
 10  is_auction     6022 non-null   bool          
 11  is_openspace   6022 non-null   bool          
 12  is_roof        6022 non-null   bool          
 13  is_SNT         6022 non-null   bool          
 14  price          6022 non-null   int64         
 15  priceM         6022 n

In [23]:
data_cian.isnull().any()

title            False
obj_name         False
adr              False
nrooms           False
floor            False
nfloors          False
area             False
is_studio        False
is_apartment     False
is_part          False
is_auction       False
is_openspace     False
is_roof          False
is_SNT           False
price            False
priceM           False
is_last_floor    False
cian_url         False
description      False
cian_page        False
ts               False
place            False
dtype: bool

In [24]:
# data_cian
# data_cian.query('nrooms==0')
# data_cian[ data_cian['title'].str.len()<1 ]
# data_cian[ data_cian['obj_name'].str.len()>0 ][['title','obj_name',]] # .to_csv('tmp/cian_title.csv',sep='\t')|

----

In [25]:
data = pd.concat([data_avito,data_cian]).reset_index(drop=True) 
print(len(data))
assert len(data)>0

24481


## обновляем таблицу адресов

In [34]:
# loc_file_path = 'data/location.pkl'# таблица адресов
# loc=pd.read_pickle(loc_file_path)
# loc = loc.fillna(False).query('~truncated').reset_index(drop=True)
# loc.to_pickle(loc_file_path)

In [35]:
from lib.locator import AddressTransformer

data['adr'] = AddressTransformer().transform( data['adr'] )

places = { # поле 'place' и название города
#'bahchisaray': 'Бахчисарай,',
#      'sudak': 'Судак,',
 'sevastopol': 'Севастополь,',
}

for k,v in places.items():
    print(k,v) # докидываем в строку адреса название города если его нет
    f =  (data['place']==k) & (~data['adr'].str.match(v))
    data.loc[f,'adr'] = v +' '+ data.loc[f,'adr']

sevastopol Севастополь,


In [36]:
from lib.locator import LocationUpdater
# from lib.locator import AddressTransformerSev
from lib.locator import GeocoderYandex
from yandex_keys import keys

loc_file_path = 'data/location.pkl'# таблица адресов

loc = LocationUpdater(
        locator=GeocoderYandex(keys[2]),
    ).transform(
        adr=data['adr'],
        loc=pd.read_pickle(loc_file_path),
        show_pbar=True,
    )

[INFO    ] 2022-09-15 23:09:59 | LocationUpdater: 10034 addresses in location table
[INFO    ] 2022-09-15 23:09:59 | LocationUpdater: 9821 addresses total
[INFO    ] 2022-09-15 23:09:59 | LocationUpdater: 9821 addresses defined
[INFO    ] 2022-09-15 23:09:59 | LocationUpdater: 0 addresses undefined


In [37]:
# from lib.locator import LocationUpdater

# loc_file_path = 'data/location.pkl'# таблица адресов

# loc = LocationUpdater().transform( по умолчанию пользуем OSM geocoder 
#         adr=data['adr'],
#         loc=pd.read_pickle(loc_file_path),
#         show_pbar=True,
#     )

In [38]:
loc.sample(3)

,adr,latitude,longitude,truncated
4794,"Античный проспект , 64 к1",44.60,33.43,False
3408,"Севастополь, садоводческое товарищество Лукулл",44.84,33.56,False
2457,"Севастополь, район Гагаринский, Омега, улица Челнокова, 14/20",44.59,33.43,False


In [39]:
assert len(loc)>0
loc.to_pickle(loc_file_path)

## дополняем данные геометкой

In [40]:
data = data.merge(loc[['adr','latitude','longitude',]],on=['adr'],how='left')

print('всего записей:', len( data) )
print('записей без геометки:', len( data[ data['latitude'].isnull() ] ) )

всего записей: 24487
записей без геометки: 0


In [41]:
assert len(data)>0
data.to_pickle(result_file_path)

In [42]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 24487 entries, 0 to 24486
Data columns (total 26 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   avito_id       18465 non-null  float64       
 1   title          24487 non-null  object        
 2   price          24487 non-null  int64         
 3   adr            24487 non-null  object        
 4   description    24487 non-null  object        
 5   obj_name       24487 non-null  object        
 6   ts             24487 non-null  datetime64[ns]
 7   avito_page     18465 non-null  float64       
 8   place          24487 non-null  object        
 9   nrooms         24487 non-null  int64         
 10  floor          24487 non-null  int64         
 11  nfloors        24487 non-null  int64         
 12  area           24487 non-null  float64       
 13  is_studio      24487 non-null  bool          
 14  is_apartment   24487 non-null  bool          
 15  is_part        2448